In [2]:
import numpy as np

In [132]:
class Mapping(object):
    
    def __init__(self, mapping_level, use_proba):
        self.mapping_level = mapping_level # This cn be "pixel" or "object".
        self.use_proba = use_proba # This can be True if the classifier is computed using the probabilities, and False otherwise.
        
        
    ## Maybe add something to pre-process the dataset.
        
        
    def train(self, train_samples, train_labels, to_save = False, save_file = 'mapping_output.npy'):
       
        ### Create the matrices that we will use for future predictions.
        
        #self.object_proba = np.array((train_samples.shape[1], 1))
        (self.privacy_classes, counts) = np.unique(train_labels, return_counts=True)
        #self.privacy_class_proba = np.array((len(self.privacy_classes), 1))
        self.object_knowing_privacy = np.zeros((train_samples.shape[1], self.privacy_classes.shape[0]))
        self.privacy_knowing_object = np.array((len(self.privacy_classes), train_samples.shape[1])) # Maybe only this one should be saved.
        
        
        ### Populate the matrices
        
        nb_samples = train_samples.shape[0]
        # Get the object probabilities.
        self.object_proba = np.sum(train_samples, axis=0) / nb_samples
    
        self.privacy_class_proba = np.asarray(counts)
        
        for i in range(0, train_samples.shape[1]):
            for j in range(0, self.privacy_classes.shape[0]):
                # Compute how many times we have object i as privacy j
                nb_obj_priv = 0
                for row in range(0, train_labels.shape[0]):
                    if train_labels[row] == self.privacy_classes[j]:
                        nb_obj_priv += train_samples[row, i]
                self.object_knowing_privacy[i, j] = nb_obj_priv / self.privacy_class_proba[j]
        self.privacy_class_proba = self.privacy_class_proba / nb_samples
        inverse_mat = np.expand_dims(np.reciprocal(self.object_proba), axis=1)
        repeat_inverse_object = np.repeat(inverse_mat, len(self.privacy_classes), axis = 1)
        repeat_privacy = np.repeat(np.expand_dims(self.privacy_class_proba, axis=0), train_samples.shape[1], axis=0)
        self.privacy_knowing_object = np.multiply(np.multiply(self.object_knowing_privacy, repeat_privacy), repeat_inverse_object)
        
        # SAve into a file for future use.
        if to_save:
            np.save(save_file, self.privacy_knowing_object) 
            np.save('classes_' + save_file, self.privacy_classes) 
        
    def evaluate(self, test_samples, test_labels):
        print("TODO")
        
    def predict(self):
        print("TODO")

In [133]:
# Test training on a toy example
train_samples = np.array([[0.7, 0.3, 0, 0], [0, 0, 1, 0], [0.8, 0, 0, 0.2]])
train_labels = np.array([1, 1, 2])

In [134]:
print(train_samples)
print(train_labels)

[[0.7 0.3 0.  0. ]
 [0.  0.  1.  0. ]
 [0.8 0.  0.  0.2]]
[1 1 2]


In [135]:
map = Mapping('object', True)

In [136]:
map.train(train_samples, train_labels)

object proba
[0.5        0.1        0.33333333 0.06666667]
privacy proba
(2,)
[[0.35 0.8 ]
 [0.15 0.  ]
 [0.5  0.  ]
 [0.   0.2 ]]
inverse  (4, 1)
Inverse object:  [[ 2.  2.]
 [10. 10.]
 [ 3.  3.]
 [15. 15.]]
REpeat privacy  (4, 2)
[[0.46666667 0.53333333]
 [1.         0.        ]
 [1.         0.        ]
 [0.         1.        ]]
TODO
